# Importing libraries

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
 
from sklearn.linear_model import LogisticRegression

from numpy import log,dot,exp,shape

from sklearn.impute import KNNImputer 

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense


# Reading dataset

In [2]:
df = pd.read_csv('onehr-data.csv')
df_original = pd.read_csv('onehr-data.csv')

# Infromation about dataset including size, and statistical infromation

In [3]:
print(df.shape)
df.describe()

(2536, 74)


,result
count,2536.000000
mean,0.028785
std,0.167236
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


# Show dataset

In [4]:
df.head()

,Date,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,1/1/1998,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,?,?,?,?,?,?,?,?,0.58,0.0


# Remove Date column (It isn't useful for prediction)

In [5]:
df.drop(columns='Date', inplace=True)
df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,?,?,?,?,?,?,?,?,0.58,0.0


In [6]:
df.replace(to_replace='?', value=np.nan, inplace=True)
df.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0.0


In [7]:
df_cc=df.dropna()

In [8]:
corr = df_cc.astype('float64').corr()
corr.style.background_gradient(cmap='coolwarm')

In [9]:
df1 = df[df['result']==0]
df2 = df[df['result']==1]

In [10]:
df1.head()

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58,0.0


In [11]:
df2.head() 

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
119,0,0.6,0.2,0.3,0.6,1,1.3,1.9,1.3,0.4,...,0.22,17.14,5.87,5745,16.4,43.5,10145,-10,0,1.0
126,2.1,1.8,1.4,1.5,3.2,2.6,NaN,3.2,3.2,2.8,...,0.18,21.59,-6.76,5840,0.9,37.5,10065,-20,0,1.0
130,1.3,1.1,0.9,0.7,0.6,0.4,0.7,2,2.3,3.6,...,0.14,24.08,-6.15,5835,3.6,36.4,10110,45,0,1.0
137,0,0.2,0,0.5,0.1,0.4,0.5,0.6,1.3,1.7,...,0.59,5.38,-2.8,5935,5.7,36.4,10190,-10,0,1.0
148,1.5,1.3,0.8,0.9,0.9,1,1.1,2.5,2.2,1.4,...,0.19,3.57,-2,5890,22.95,49.35,10140,5,0,1.0


In [12]:
df1_cc = df1.dropna()
df2_cc = df2.dropna()

In [13]:
df1_cc

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
0,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,2.3,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
6,3.7,3.2,3.8,5.1,6,7,6.3,6.4,6.3,5.4,...,0.84,6.86,25.6,5695,26.75,48.45,10040,-80,0.18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,0.3,0.4,0.5,0.5,0.2,0.3,0.4,0.4,1.3,2.2,...,0.07,7.93,-4.41,5800,-25.6,21.8,10295,65,0,0.0
2532,1,1.4,1.1,1.7,1.5,1.7,1.8,1.5,2.1,2.4,...,0.04,5.95,-1.14,5845,-19.4,19.1,10310,15,0,0.0
2533,0.8,0.8,1.2,0.9,0.4,0.6,0.8,1.1,1.5,1.5,...,0.06,7.8,-0.64,5845,-9.6,35.2,10275,-35,0,0.0
2534,1.3,0.9,1.5,1.2,1.6,1.8,1.1,1,1.9,2,...,0.25,7.72,-0.89,5845,-19.6,34.2,10245,-30,0.05,0.0


In [14]:
df2_cc

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,result
119,0,0.6,0.2,0.3,0.6,1,1.3,1.9,1.3,0.4,...,0.22,17.14,5.87,5745,16.4,43.5,10145,-10,0,1.0
130,1.3,1.1,0.9,0.7,0.6,0.4,0.7,2,2.3,3.6,...,0.14,24.08,-6.15,5835,3.6,36.4,10110,45,0,1.0
137,0,0.2,0,0.5,0.1,0.4,0.5,0.6,1.3,1.7,...,0.59,5.38,-2.8,5935,5.7,36.4,10190,-10,0,1.0
148,1.5,1.3,0.8,0.9,0.9,1,1.1,2.5,2.2,1.4,...,0.19,3.57,-2,5890,22.95,49.35,10140,5,0,1.0
208,0.9,1.1,0.8,1.4,0.8,0.7,2,2.9,2.2,1.7,...,0.19,-4.96,-1.06,5965,23.2,42.4,10180,5,0,1.0
209,1.7,0.8,1.1,1.1,0.1,0.3,0.3,2.1,3.5,3,...,0.17,-4.33,-2.78,5955,35.65,46.4,10165,-15,0,1.0
222,0.9,0.6,0.4,0.8,0.2,1.8,1.7,1,1.3,1.5,...,0.84,6.61,2.14,5860,33.05,44.7,10110,-40,0,1.0
237,1.1,1.8,1.6,1.2,1.3,1.5,1.5,0.8,2.1,2.3,...,0.37,0.47,2.64,5910,24.9,48,10130,-5,0.51,1.0
241,0.2,0.4,0.2,0.2,0.2,0.4,0.6,1.4,1.4,0.4,...,0.14,5.54,-8.58,5850,16,36.7,10070,10,0,1.0
242,0.7,1,0.2,0.4,0.1,0.2,0.2,0.7,1.2,2.1,...,0.05,-3.85,-4.59,5885,7.4,36.7,10105,35,0,1.0


In [15]:
df1_cc.isnull().values.any()

False

In [16]:
df2_cc.isnull().values.any()

False

# Handling missing values

In [17]:
corr1 = df1_cc.astype('float64').corr()
corr1.style.background_gradient(cmap='coolwarm')

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
C:\Users\Administrator\anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


In [18]:
corr2 = df2_cc.astype('float64').corr()
corr2.style.background_gradient(cmap='coolwarm')

In [19]:
# Splitting the dataset into feature(X) and target(Y)
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [20]:
imputer = KNNImputer(n_neighbors=3, weights="uniform")
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#df.dropna(inplace=True,axis=0)
#df.interpolate(method='slinear',limit_direction ='both')

In [21]:
# check if NaN values exist
if np.nan in df['T_PK'].values.tolist():
  print("NaN values found")
else:
  print("No NaN values found")

NaN values found


In [22]:
X = imputer.fit_transform(x)

# Kfold cross validation

In [23]:
kf = KFold(n_splits=3, shuffle=True, random_state=1)

# Logistic regression with inbuilt functions of python

In [24]:
def standardize(X_tr):
    for i in range(X_tr.shape[1]):
        X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])
def evaluation(y,y_hat):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(y)):
        if y[i] == 1 and y_hat[i] == 1:
            tp += 1
        elif y[i] == 1 and y_hat[i] == 0:
            fn += 1
        elif y[i] == 0 and y_hat[i] == 1:
            fp += 1
        elif y[i] == 0 and y_hat[i] == 0:
            tn += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    f1_score = 2*precision*recall/(precision+recall)
    return precision,accuracy
class my_LogisticRegression() :
    def __init__( self, learning_rate, iterations ) :        
        self.learning_rate = learning_rate        
        self.iterations = iterations
    # Function for model training    
    def fit( self, X, Y ) :        
        # no_of_training_examples, no_of_features        
        self.m, self.n = X.shape        
        # weight initialization        
        self.W = np.zeros( self.n )        
        self.b = 0        
        self.X = X        
        self.Y = Y
        # gradient descent learning
        for i in range( self.iterations ) :            
            self.update_weights()            
        return self
    # Helper function to update weights in gradient descent
    def update_weights( self ) :           
        A = 1 / ( 1 + np.exp( - ( self.X.dot( self.W ) + self.b ) ) )
        # calculate gradients        
        tmp = ( A - self.Y.T )        
        tmp = np.reshape( tmp, self.m )        
        dW = np.dot( self.X.T, tmp ) / self.m         
        db = np.sum( tmp ) / self.m 
        # update weights    
        self.W = self.W - self.learning_rate * dW    
        self.b = self.b - self.learning_rate * db
        return self
    # Hypothetical function  h( x ) 
    def predict( self, X ) :    
        Z = 1 / ( 1 + np.exp( - ( X.dot( self.W ) + self.b ) ) )        
        Y = np.where( Z > 0.5, 1, 0 )        
        return Y

In [25]:
tr_accuracy = list()
tr_precision = list()
te_accuracy = list()
te_precision = list()
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    standardize(X_train)
    standardize(X_test)
    obj1 = my_LogisticRegression(0.123,41000)
    model= obj1.fit(X_train,y_train)
    y_pred = obj1.predict(X_test)
    y_tr = obj1.predict(X_train)
    #Let's see evalutation parameters for training and testing data
    evaluation_tr_precision,evaluation_tr_accuracy = evaluation(y_tr,y_train)
    evaluation_te_precision,evaluation_te_accuracy= evaluation(y_test,y_pred)
    tr_precision.append(evaluation_tr_precision)
    tr_accuracy.append(evaluation_tr_accuracy)
    te_precision.append(evaluation_te_precision)
    te_accuracy.append(evaluation_te_accuracy)
print("Precision of train sets:",tr_precision)
print("Accuracy of train sets:",tr_accuracy)
print("Precision of test sets:",te_precision)
print("Accuracy of test sets:",te_accuracy)
print(np.mean(tr_precision)*100)
print(np.mean(tr_accuracy)*100)
print(np.mean(te_precision)*100)
print(np.mean(te_accuracy)*100)

Precision of train sets: [0.25, 0.4090909090909091, 0.2962962962962963]
Accuracy of train sets: [0.9763313609467456, 0.9828503843879361, 0.9745712596096984]
Precision of test sets: [0.4, 0.23529411764705882, 0.2]
Accuracy of test sets: [0.9692671394799054, 0.9550295857988166, 0.9704142011834319]
31.846240179573513
97.79176683147934
27.843137254901958
96.49036421540514


In [ ]:
tr_accuracy_cnn = list()
tr_loss_cnn = list()
te_accuracy_cnn = list()
te_loss_cnn = list()
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # determine the number of input features
    n_features = X_train.shape[1]
    # define model
    model = Sequential()
    model.add(Dense(20, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
    model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # fit the model
    model.fit(X_train, y_train, epochs=10, batch_size=1500, verbose=0)
    # evaluate the model
    tr_loss,tr_acc = model.evaluate(X_train, y_train, verbose=0)
    te_loss,te_acc = model.evaluate(X_test, y_test, verbose=0)
    tr_loss_cnn.append(tr_loss)
    tr_accuracy_cnn.append(tr_acc)
    te_loss_cnn.append(te_loss)
    te_accuracy_cnn.append(te_acc)

print("Accuracy of train sets:",tr_accuracy_cnn)
print("Accuracy of test sets:",te_accuracy_cnn)
print(np.mean(tr_accuracy_cnn)*100)
print(np.mean(te_accuracy_cnn)*100)